In [1]:
from sklearn.datasets import *
import numpy as np
import pandas as pd
import os
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk import PorterStemmer as Stemmer
import string
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
digit = load_digits()
dig = pd.DataFrame(digit['data'][0:1700])
print(dig.head())
print(digit.keys())

In [ ]:
plt.imshow(digit['images'][8], cmap='Greys_r')
plt.show()

In [ ]:
train_x = digit.data # input variables
train_y = digit.target # les étiquettes (output variable)
#découpage du jeu de données (0.25 pour indiquer 25%)
x_train,x_test,y_train,y_test=train_test_split(train_x,train_y,test_size=0.25)

In [ ]:
### Train
KNN = KNeighborsClassifier(7) # on utilise 7 voisins
KNN.fit(x_train, y_train)
print(KNN.score(x_test,y_test))

In [ ]:
df = pd.read_csv('C:/DB/DB/spam.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['label', 'message']
print(df.head())

In [ ]:
nltk.download('stopwords')

In [ ]:
def process(text):
    # lowercase it
    text = text.lower()
    # remove punctuation
    text = ''.join([t for t in text if t not in string.punctuation])
    # remove stopwords
    text = [t for t in text.split() if t not in stopwords.words('english')]
    # stemming
    st = Stemmer()
    text = [st.stem(t) for t in text]
    # return token list
    return text

In [ ]:
# Testing
print(process('It\'s holiday and we are playing cricket. Jeff is playing very well!!!'))
# Test avec notre dataset
print(df['message'][:20].apply(process))

In [ ]:
tfidfv = TfidfVectorizer(analyzer=process)
data = tfidfv.fit_transform(df['message'])
mess = df.iloc[2]['message']
print(mess)
print(tfidfv.transform([mess]))
#visualiser
j = tfidfv.transform([mess]).toarray()[0]
print('index\tidf\ttfidf\tterm')
for i in range(len(j)):
    if j[i] != 0:
        print(i, format(tfidfv.idf_[i], '.4f'), format(j[i], '.4f'),
tfidfv.get_feature_names_out()[i],sep='\t')

In [ ]:
########### TRAIN ############
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
spam_filter = Pipeline([
('vectorizer', TfidfVectorizer(analyzer=process)), # messages to weighted TFIDF score
('classifier', MultinomialNB()) # train on TFIDF vectors with NB
])
x_train, x_test, y_train, y_test = train_test_split(df['message'],
df['label'], test_size=0.20, random_state = 21)
spam_filter.fit(x_train, y_train)
######## TEST ##############
predictions = spam_filter.predict(x_test)
count = 0
for i in range(len(y_test)):
    if y_test.iloc[i] != predictions[i]:
        print(x_test.iloc[i])
        count += 1
print('Total number of test cases', len(y_test))
print('Number of wrong of predictions', count)

In [ ]:
#Calculez Rappel Précision F1 score
from sklearn.metrics import classification_report
print(classification_report(predictions, y_test))